### 1.基本分词函数与用法

jieba.cut以及jieba.cut_for_search返回的结构都是一个可迭代的generator，可以使用for循环来获得分词后得到的每一个词语（Unicode）

**jieba.cut** 方法接受三个输入参数：
* 需要分词的字符串
* cut_all参数用来控制是否采用全模式
* HMM参数用来控制是否使用HMM模型

**jieba.cut_for_search** 方法接受两个参数
* 需要分词的字符串
* 是否使用HMM模型

该方法适合用于搜索引擎构建倒排索引的分词，粒度比较细

In [ ]:
import jieba

seg_list = jieba.cut("我在学习自然语言处理", cut_all=True)
print(seg_list)
print("Full Mode: " + "/".join(seg_list))  # 全模式

seg_list = jieba.cut("我在学习自然语言处理", cut_all=False)
print("Default Mode: " + "/".join(seg_list)) # 精确模式

seg_list = jieba.cut("我在学习自然语言处理")  # 默认是精确模式
print(", ".join(seg_list))

seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在哈佛大学深造")  # 搜索引擎模式
print(", ".join(seg_list))

**jieba.lcut** 以及 **jieba.lcut_for_search** 直接返回 list

In [ ]:
result_lcut = jieba.lcut("小明硕士毕业于中国科学院计算所，后在哈佛大学深造")
print(result_lcut)
print(" ".join(result_lcut))
print(" ".join(jieba.lcut_for_search("小明硕士毕业于中国科学院计算所，后在哈佛大学深造")))

**添加用户自定义词典**

很多时候我们需要针对自己的场景进行分词，会有一些领域内的专有词汇。
* 1.可以用jieba.load_userdict(file_name)加载用户字典
* 2.少量的词汇可以自己用下面方法手动添加：
    * 用 add_word(word, freq=None, tag=None) 和 del_word(word) 在程序中动态修改词典
    * 用 suggest_freq(segment, tune=True) 可调节单个词语的词频，使其能（或不能）被分出来。

In [ ]:
print('/'.join(jieba.cut('如果放到旧字典中将出错。', HMM=False)))

In [ ]:
# 调整'中', '将'，将'中将'分出来 
jieba.suggest_freq(('中', '将'), True)

In [ ]:
print('/'.join(jieba.cut('如果放到旧字典中将出错。', HMM=False)))

### 2.关键词提取

**基于 TF-IDF 算法的关键词抽取**

import jieba.analyse

* jieba.analyse.extract_tags(sentence, topK=20, withWeigth=False,allowPOS=())
    * sentence为待提取的文本
    * topK为返回几个TF/IDF权重最大的关键词，默认值为20
    * withWeight 为是否一并返回关键词权重值，默认值为 False
    * allowPOS 仅包括指定词性的词，默认值为空，即不筛选

In [ ]:
import jieba.analyse as analyse
lines = open('./data/NBA.txt', encoding='utf-8').read()
print("  ".join(analyse.extract_tags(lines, topK=20, withWeight=False, allowPOS=())))

In [ ]:
lines = open('./data/西游记.txt', encoding='utf-8').read()
print("  ".join(analyse.extract_tags(lines, topK=20, withWeight=False, allowPOS=())))

**关于TF-IDF算法的关键词抽取补充**
* 关键词提取所使用逆向文件频率（IDF）文本语料库可以切换成自定义语料库的路径
    * 用法： jieba.analyse.set_idf_path(file_name) # file_name为自定义语料库的路径
* 关键词提取所使用停止词（Stop Words）文本语料库可以切换成自定义语料库的路径
    * 用法： jieba.analyse.set_stop_words(file_name) # file_name为自定义语料库的路径

**基于TextRank算法的关键词抽取**
* jieba.analyse.textrank(sentence, topK=20, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v')) 直接使用，接口相同，注意默认过滤词性。
    * ns表示地名、n表示名词、vn表示名动词、v表示动词
* jieba.analyse.TextRank() 新建自定义 TextRank 实例

基本思想:
* 将待抽取关键词的文本进行分词
* 以固定窗口大小(默认为5，通过span属性调整)，词之间的共现关系，构建图
* 计算图中节点的PageRank，注意是无向带权图

In [ ]:
import jieba.analyse as analyse
lines = open('./data/NBA.txt', encoding='utf-8').read()
print("  ".join(analyse.textrank(lines, topK=20, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v'))))
print("---------------------分割线----------------")
print("  ".join(analyse.textrank(lines, topK=20, withWeight=False, allowPOS=('ns', 'n'))))

In [ ]:
lines = open('./data/西游记.txt', encoding='utf-8').read()
print("  ".join(analyse.textrank(lines, topK=20, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v'))))

### 词性标注
* jieba.posseg.POSTokenizer(tokenizer=None) 新建自定义分词器，tokenizer 参数可指定内部使用的 jieba.Tokenizer 分词器。jieba.posseg.dt 为默认词性标注分词器。
* 标注句子分词后每个词的词性，采用和 ictclas 兼容的标记法。

In [ ]:
import jieba.posseg as pseg
words = pseg.cut("我正在学习自然语言处理")
for word, flag in words:
    print('%s %s' % (word, flag))

### 并行分词
原理：将目标文本按行分隔后，把各行文本分配到多个 Python 进程并行分词，然后归并结果，从而获得分词速度的可观提升 基于 python 自带的 multiprocessing 模块，目前暂不支持 Windows
用法：
    jieba.enable_parallel(4) # 开启并行分词模式，参数为并行进程数
    jieba.disable_parallel() # 关闭并行分词模式
注意：并行分词仅支持默认分词器 jieba.dt 和 jieba.posseg.dt。

In [ ]:
import sys
import time
import jieba

jieba.enable_parallel()
content = open('./data/西游记.txt',"r", encoding='utf-8').read()
t1 = time.time()
words = "/ ".join(jieba.cut(content))
t2 = time.time()
tm_cost = t2-t1
print('并行分词速度为 %s bytes/second' % (len(content)/tm_cost))

jieba.disable_parallel()
content = open('./data/西游记.txt',"r", encoding='utf-8').read()
t1 = time.time()
words = "/ ".join(jieba.cut(content))
t2 = time.time()
tm_cost = t2-t1
print('非并行分词速度为 %s bytes/second' % (len(content)/tm_cost))

### Tokenize：返回词语在原文的起止位置

In [ ]:
print("这是默认模式的tokenize")
result = jieba.tokenize('自然语言处理非常有用')
for tk in result:
    print("%s\t\t start: %d \t\t end:%d" % (tk[0],tk[1],tk[2]))
    
print("\n-----------分割线------------\n")
print("这是搜索模式的tokenize")
result = jieba.tokenize('自然语言处理非常有用', mode='search')
for tk in result:
    print("%s\t\t start: %d \t\t end:%d" % (tk[0],tk[1],tk[2]))